In [2]:
import pandas as pd

In [3]:
df_orders = pd.read_csv(r'C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/raw/orders.csv')

df_orders

,order_id,order_date,customer_id,region_id,ship_mode,payment_method,delivery_time_days,shipping_cost,order_priority
0,O202311050000,11/5/2023,CUST_2209,R03,Standard,Bancontact,6.0,15.91,Low
1,O202505200001,5/20/2025,CUST_2123,R07,express,Visa,3.0,5.40,Low
2,O202308170002,8/17/2023,CUST_1491,R08,Express,visa,2.0,0.94,Urgent
3,O202305220003,5/22/2023,CUST_2065,R02,Economy,SEPA,9.0,8.98,MEDIUM
4,O202510250004,10/25/2025,CUST_2069,R07,STANDARD,SEPA,6.0,2.16,Medium
...,...,...,...,...,...,...,...,...,...
4995,O202404254995,4/25/2024,CUST_1315,R06,express,PayPal,2.0,14.47,Urgent
4996,O202502254996,2/25/2025,CUST_2089,R01,Economy,SEPA,9.0,0.65,High
4997,O202411074997,11/7/2024,CUST_2140,R06,Economy,visa,7.0,5.23,Low
4998,O202302174998,2/17/2023,CUST_1190,R06,Overnight,Mastercard,1.0,15.75,MEDIUM


In [6]:
# ============================================
# LOAD REGION MAPPING FROM REGIONS NOTEBOOK
# ============================================

import pickle
with open('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/region_id_mapping.pkl', 'rb') as f:
    region_id_mapping = pickle.load(f)

df_orders_clean = df_orders.copy()

# CRITICAL: Apply region mapping BEFORE other cleaning
print(f"\nBefore mapping:")
print(df_orders_clean['region_id'].value_counts())

df_orders_clean['region_id'] = df_orders_clean['region_id'].replace(region_id_mapping)

print(f"\nAfter mapping:")
print(df_orders_clean['region_id'].value_counts())

# ============================================
# LOAD CUSTOMER MAPPING FROM CUSTOMERS NOTEBOOK
# ============================================

customer_mapping_df = pd.read_csv('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/customer_id_mapping.csv')
customer_id_mapping = dict(zip(customer_mapping_df['old_customer_id'], 
                              customer_mapping_df['new_customer_id']))
print(f"✓ Loaded customer mapping: {customer_id_mapping}")

# IMPORTANT: Update customer_ids based on merge mapping (BEFORE other cleaning). Both mappings should be applied before other cleaning.

df_orders_clean['customer_id'] = df_orders_clean['customer_id'].replace(customer_id_mapping)

# Standardize ship_mode
ship_mode_mapping = {
    'express': 'Express',
    'EXPRESS': 'Express',
    'Express': 'Express',
    'standard': 'Standard',
    'STANDARD': 'Standard',
    'Standard': 'Standard',
    'overnight': 'Overnight',
    'Overnight': 'Overnight',
    'economy': 'Economy',
    'Economy': 'Economy'
}
df_orders_clean['ship_mode'] = df_orders_clean['ship_mode'].replace(ship_mode_mapping)

# Standardize payment_method
payment_mapping = {
    'visa': 'Visa',
    'VISA': 'Visa',
    'mastercard': 'Mastercard',
    'MASTERCARD': 'Mastercard',
    'paypal': 'PayPal',
    'PAYPAL': 'PayPal',
    'bancontact': 'Bancontact',
    'ideal': 'iDEAL',
    'sepa': 'SEPA'
}
df_orders_clean['payment_method'] = df_orders_clean['payment_method'].replace(payment_mapping)

# Standardize order_priority
priority_mapping = {
    'low': 'Low',
    'LOW': 'Low',
    'medium': 'Medium',
    'MEDIUM': 'Medium',
    'high': 'High',
    'HIGH': 'High',
    'urgent': 'Urgent',
    'URGENT': 'Urgent'
}
df_orders_clean['order_priority'] = df_orders_clean['order_priority'].replace(priority_mapping)

# Impute missing delivery_time_days (2.1% NULL)
# Use median delivery time by ship_mode
df_orders_clean['delivery_time_days'] = df_orders_clean.groupby('ship_mode')['delivery_time_days'].transform(
    lambda x: x.fillna(x.median())
)

print("✓ Orders table cleaned!")
print(f"✓ Updated {len(customer_id_mapping)} customer_id references")

# Save cleaned orders
df_orders_clean.to_csv('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/orders_cleaned.csv', index=False)
print("✓ Saved: C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/orders_cleaned.csv")


Before mapping:
region_id
R06    666
R01    652
R05    630
R08    627
R04    619
R02    610
R07    599
R03    597
Name: count, dtype: int64

After mapping:
region_id
R01    1318
R04    1246
R03    1196
R05     630
R02     610
Name: count, dtype: int64
✓ Loaded customer mapping: {'CUST_2137': 'CUST_1439', 'CUST_2220': 'CUST_1715', 'CUST_1646': 'CUST_1715', 'CUST_1837': 'CUST_1715', 'CUST_1775': 'CUST_1280', 'CUST_1989': 'CUST_1661', 'CUST_1474': 'CUST_1661', 'CUST_1379': 'CUST_1791', 'CUST_1321': 'CUST_1791', 'CUST_1391': 'CUST_1791', 'CUST_2167': 'CUST_1575', 'CUST_2180': 'CUST_1575', 'CUST_2129': 'CUST_1575', 'CUST_2131': 'CUST_1882', 'CUST_2061': 'CUST_1882', 'CUST_1078': 'CUST_1347', 'CUST_1042': 'CUST_1347', 'CUST_1116': 'CUST_1477', 'CUST_1193': 'CUST_1718', 'CUST_1145': 'CUST_1563', 'CUST_1571': 'CUST_1852', 'CUST_1620': 'CUST_1852', 'CUST_1006': 'CUST_1852', 'CUST_1946': 'CUST_1045', 'CUST_2005': 'CUST_1678', 'CUST_2202': 'CUST_1708', 'CUST_2069': 'CUST_2066', 'CUST_1809': 'CUS